## Using BeautifulSoup/Selenium to Scrape BaseballSavant

First, importing needed packages:

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline
rcParams['figure.figsize'] = 20,10
import numpy as np
import glob
from scipy import stats
from bs4 import BeautifulSoup
import requests
import re
from IPython.core.display import display, HTML    # make sure Jupyter knows to display it as HTML

Importing selenium and setting chrome driver:

In [3]:
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [4]:
#Data from 2015-2020, players with at least 200 PA in that given season:
savant_url = 'https://baseballsavant.mlb.com/leaderboard/custom?year=2020,2019,2018,2017,2016,2015&type=batter&filter=&sort=16&sortDir=desc&min=200&selections=player_age,b_total_pa,b_game,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,solidcontact_percent,hard_hit_percent,z_swing_percent,oz_swing_percent,pull_percent,straightaway_percent,opposite_percent,groundballs_percent,flyballs_percent,linedrives_percent,sprint_speed,&chart=true&x=player_age&y=player_age&r=no&chartType=beeswarm'

If using wOBA, insert this link above:
https://baseballsavant.mlb.com/leaderboard/custom?year=2020,2019,2018,2017,2016,2015&type=batter&filter=&sort=16&sortDir=desc&min=200&selections=player_age,b_total_pa,b_game,woba,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,solidcontact_percent,hard_hit_percent,z_swing_percent,oz_swing_percent,pull_percent,straightaway_percent,opposite_percent,groundballs_percent,flyballs_percent,linedrives_percent,sprint_speed,&chart=true&x=player_age&y=player_age&r=no&chartType=beeswarm

In [5]:
driver = webdriver.Chrome(chromedriver)
driver.get(savant_url)

In [6]:
driver.page_source[:1000]

'<html lang="en_US" class=" device-desktop"><head>\n    <title>Statcast Custom Leaderboards | baseballsavant.com </title>\n\n    <!-- meta meta tag -->\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=Edge">\n    <meta http-equiv="Cache-Control" content="no-cache">\n    <meta http-equiv="Pragma" content="no-cache">\n    <meta http-equiv="Expires" content="-1">\n    <meta name="keywords" content="Baseball Savant">\n    <!-- -->\n    <meta property="og:title" content="Statcast Custom Leaderboards">\n    <meta itemprop="name" content="Statcast Custom Leaderboards baseballsavant.com">\n    \n    <meta property="og:site_name" content="baseballsavant.com">\n    <meta property="og:type" content="website">\n    \n    <meta property="og:image" content="https://baseballsavant.mlb.com/site-core/images/statcast-logo-gcp-social.png">\n    <meta itemprop="image" content="https://baseballsavant.mlb.com/site-core/images/statcast-logo-gcp-social.png">\n     <meta name="tw

In [7]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [8]:
headers = soup.find('div', id='sortable_stats').find_all('th')

Finding Table Rows:

In [9]:
columns = [col.get_text() for col in headers]

In [10]:
print(columns)

['Rk.', 'Player', 'Year', 'Age', 'PA', 'G', 'Avg EV (MPH)', 'Avg LA (°)', 'Barrel%', 'Solid Contact %', 'Hard Hit %', 'Zone Swing %', 'Out of Zone Swing %', 'Pull %', 'Straight Away %', 'Oppo %', 'GB%', 'FB%', 'LD %', 'Sprint Speed']


In [11]:
savant_df = pd.DataFrame(columns=columns)
savant_df.head()

,Rk.,Player,Year,Age,PA,G,Avg EV (MPH),Avg LA (°),Barrel%,Solid Contact %,Hard Hit %,Zone Swing %,Out of Zone Swing %,Pull %,Straight Away %,Oppo %,GB%,FB%,LD %,Sprint Speed


Finding Stats:

In [12]:
stats_test = soup.find('div', id='sortable_stats').find_all('tr')

In [13]:
stats = [col.get_text() for col in stats_test]

In [14]:
stat_lines = soup.find('div', id='sortable_stats').find_all('tr')
compiled_stats = []
for line_item in stat_lines:
    ind_stat_line = [item.get_text() for item in line_item.find_all('td')]
    compiled_stats.append(ind_stat_line)
    final_stats = compiled_stats[1:]

Pulling links to player pages:

In [15]:
player_links = soup.find('div', id='sortable_stats').find_all('tr')
compiled_links = []
for line_item in player_links:
    ind_link = [item.get("href") for item in line_item.find_all('a')]
    compiled_links.append(ind_link)
    final_link = compiled_links[1:]

In [16]:
print(final_link)

[['/savant-player/457454'], ['/savant-player/572114'], ['/savant-player/608336'], ['/savant-player/572761'], ['/savant-player/572761'], ['/savant-player/457803'], ['/savant-player/608336'], ['/savant-player/519346'], ['/savant-player/656555'], ['/savant-player/624415'], ['/savant-player/595885'], ['/savant-player/608336'], ['/savant-player/435063'], ['/savant-player/545361'], ['/savant-player/572122'], ['/savant-player/641598'], ['/savant-player/519346'], ['/savant-player/519390'], ['/savant-player/457803'], ['/savant-player/474832'], ['/savant-player/545350'], ['/savant-player/455139'], ['/savant-player/592206'], ['/savant-player/572761'], ['/savant-player/594807'], ['/savant-player/545361'], ['/savant-player/448801'], ['/savant-player/656555'], ['/savant-player/593934'], ['/savant-player/474892'], ['/savant-player/594807'], ['/savant-player/455139'], ['/savant-player/457759'], ['/savant-player/572761'], ['/savant-player/472528'], ['/savant-player/446263'], ['/savant-player/656555'], 

Building DataFrame:

In [17]:
savant_df = pd.DataFrame(final_stats, columns=columns)
num_cols = ['Age','G','PA','Year','Avg EV (MPH)','Avg LA (°)','Barrel%','Solid Contact %','Hard Hit %','Zone Swing %','Out of Zone Swing %','Pull %','Straight Away %','Oppo %','GB%','FB%','LD %','Sprint Speed']
savant_df[num_cols] = savant_df[num_cols].apply(pd.to_numeric, errors='coerce', axis=1)
savant_df.set_index(savant_df['Rk.'], inplace=True)
savant_df.drop(columns='Rk.',inplace=True)
savant_df['Player Link'] = final_link
savant_df.head()

,Player,Year,Age,PA,G,Avg EV (MPH),Avg LA (°),Barrel%,Solid Contact %,Hard Hit %,Zone Swing %,Out of Zone Swing %,Pull %,Straight Away %,Oppo %,GB%,FB%,LD %,Sprint Speed,Player Link
Rk.,,,,,,,,,,,,,,,,,,,,
1,"Saltalamacchia, Jarrod",2016.0,31.0,292.0,92.0,89.2,21.4,11.6,6.8,35.4,65.7,23.9,44.2,34.0,21.8,31.3,43.5,19.7,25.6,[/savant-player/457454]
2,"Schimpf, Ryan",2016.0,28.0,330.0,89.0,90.3,29.7,16.7,8.0,41.4,64.7,23.4,41.4,33.9,24.7,20.1,42.0,21.8,28.2,[/savant-player/572114]
3,"Gallo, Joey",2017.0,24.0,532.0,145.0,93.3,23.0,22.1,8.3,52.2,72.7,28.2,50.2,29.2,20.6,29.2,37.9,22.1,27.8,[/savant-player/608336]
4,"Carpenter, Matt",2018.0,33.0,677.0,156.0,90.7,21.6,13.7,11.5,44.7,57.1,18.5,47.6,30.7,21.7,28.3,37.8,28.5,26.5,[/savant-player/572761]
5,"Carpenter, Matt",2017.0,32.0,622.0,145.0,89.9,22.7,8.2,8.4,37.7,55.1,14.0,45.9,31.4,22.7,29.0,37.7,26.6,26.6,[/savant-player/572761]


In [18]:
savant_df.Player.value_counts()

 Rizzo, Anthony      6
 Escobar, Eduardo    6
 Abreu, Jose         6
 Sano, Miguel        6
 Ramirez, Jose       6
                    ..
 White, Evan         1
 Crisp, Coco         1
 Butler, Joey        1
 Munoz, Yairo        1
 Slater, Austin      1
Name: Player, Length: 646, dtype: int64

In [19]:
savant_df

,Player,Year,Age,PA,G,Avg EV (MPH),Avg LA (°),Barrel%,Solid Contact %,Hard Hit %,Zone Swing %,Out of Zone Swing %,Pull %,Straight Away %,Oppo %,GB%,FB%,LD %,Sprint Speed,Player Link
Rk.,,,,,,,,,,,,,,,,,,,,
1,"Saltalamacchia, Jarrod",2016.0,31.0,292.0,92.0,89.2,21.4,11.6,6.8,35.4,65.7,23.9,44.2,34.0,21.8,31.3,43.5,19.7,25.6,[/savant-player/457454]
2,"Schimpf, Ryan",2016.0,28.0,330.0,89.0,90.3,29.7,16.7,8.0,41.4,64.7,23.4,41.4,33.9,24.7,20.1,42.0,21.8,28.2,[/savant-player/572114]
3,"Gallo, Joey",2017.0,24.0,532.0,145.0,93.3,23.0,22.1,8.3,52.2,72.7,28.2,50.2,29.2,20.6,29.2,37.9,22.1,27.8,[/savant-player/608336]
4,"Carpenter, Matt",2018.0,33.0,677.0,156.0,90.7,21.6,13.7,11.5,44.7,57.1,18.5,47.6,30.7,21.7,28.3,37.8,28.5,26.5,[/savant-player/572761]
5,"Carpenter, Matt",2017.0,32.0,622.0,145.0,89.9,22.7,8.2,8.4,37.7,55.1,14.0,45.9,31.4,22.7,29.0,37.7,26.6,26.6,[/savant-player/572761]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1889,"Jankowski, Travis",2016.0,25.0,383.0,131.0,86.2,0.8,1.3,1.3,26.9,56.9,21.4,20.2,43.3,36.6,62.2,10.1,26.5,29.5,[/savant-player/608671]
1890,"LeMahieu, DJ",2016.0,28.0,635.0,146.0,91.7,5.9,4.9,5.6,47.5,59.2,23.0,20.8,42.0,37.2,52.9,10.1,33.7,27.1,[/savant-player/518934]
1891,"Jankowski, Travis",2018.0,27.0,387.0,117.0,85.6,3.3,0.7,1.1,19.3,59.2,18.7,30.1,39.9,30.1,60.1,10.1,25.4,29.0,[/savant-player/608671]


Driver's work here is done.

In [20]:
driver.close()

### Use player links to pull position data from their individual BS page:

Making a list of players' names first, so i don't make duplicate links (for time's sake):

In [27]:
player_no_duplicate_df = pd.DataFrame(data=[savant_df['Player'],savant_df['Player Link']]).T

In [28]:
player_no_duplicate_df

,Player,Player Link
Rk.,,
1,"Saltalamacchia, Jarrod",[/savant-player/457454]
2,"Schimpf, Ryan",[/savant-player/572114]
3,"Gallo, Joey",[/savant-player/608336]
4,"Carpenter, Matt",[/savant-player/572761]
5,"Carpenter, Matt",[/savant-player/572761]
...,...,...
1889,"Jankowski, Travis",[/savant-player/608671]
1890,"LeMahieu, DJ",[/savant-player/518934]
1891,"Jankowski, Travis",[/savant-player/608671]


*1893 rows match above dataframe, with only the two needed columns.*

Next step, remove duplicates from the player column, since their links point to the same page (removing work for the computer later)

In [29]:
player_no_duplicate_df.drop_duplicates(subset='Player', inplace=True)

In [30]:
player_no_duplicate_df

,Player,Player Link
Rk.,,
1,"Saltalamacchia, Jarrod",[/savant-player/457454]
2,"Schimpf, Ryan",[/savant-player/572114]
3,"Gallo, Joey",[/savant-player/608336]
4,"Carpenter, Matt",[/savant-player/572761]
6,"Bruce, Jay",[/savant-player/457803]
...,...,...
1885,"Venable, Will",[/savant-player/461416]
1886,"Fuld, Sam",[/savant-player/453539]
1887,"Schumaker, Skip",[/savant-player/435401]


Mission accomplished, down to only 646 rows!

Now, make a list of all the links to feed into a soup generator:

In [31]:
link_list = player_no_duplicate_df['Player Link'].to_list()

In [32]:
savant_page_list = [('https://baseballsavant.mlb.com' + str(link).strip("'[]")) for link in link_list]

In [33]:
print(savant_page_list)

['https://baseballsavant.mlb.com/savant-player/457454', 'https://baseballsavant.mlb.com/savant-player/572114', 'https://baseballsavant.mlb.com/savant-player/608336', 'https://baseballsavant.mlb.com/savant-player/572761', 'https://baseballsavant.mlb.com/savant-player/457803', 'https://baseballsavant.mlb.com/savant-player/519346', 'https://baseballsavant.mlb.com/savant-player/656555', 'https://baseballsavant.mlb.com/savant-player/624415', 'https://baseballsavant.mlb.com/savant-player/595885', 'https://baseballsavant.mlb.com/savant-player/435063', 'https://baseballsavant.mlb.com/savant-player/545361', 'https://baseballsavant.mlb.com/savant-player/572122', 'https://baseballsavant.mlb.com/savant-player/641598', 'https://baseballsavant.mlb.com/savant-player/519390', 'https://baseballsavant.mlb.com/savant-player/474832', 'https://baseballsavant.mlb.com/savant-player/545350', 'https://baseballsavant.mlb.com/savant-player/455139', 'https://baseballsavant.mlb.com/savant-player/592206', 'https://

In [35]:
#Running the len just to audit/sanity check:
len(savant_page_list)

646

Defining a Function to generate the soup objects:

In [36]:
def Soup_Generator(link_list):
    '''
    Takes in a list of players and returns Beautiful Soup objects.
    '''
    response_list = [requests.get(url) for url in link_list]
    soup_list = []
    for response in response_list:
        page = response.text
        soup_list.append(BeautifulSoup(page,'html'))
    return soup_list

Find Player Position Data:

In [38]:
link_soup = Soup_Generator(savant_page_list)

KeyboardInterrupt: 

In [ ]:
position_list = []
for player in link_soup:
    position = player.find('div', style='font-size: .8rem;')
    position_list.append(str(position.text))

In [ ]:
print(position_list)

In [ ]:
final_position = []
for position in position_list:
    position = position.strip("\n")
    position = position.strip(" ")
    final_position.append(position[0:2].strip(" "))
print(final_position)

Defining function for position generation:

In [102]:
def Position_Puller(soup_list):
    '''
    Takes in a list of Beautiful Soup Objects from BasebalL Savant
    and returns each player's position.
    '''
    position_list = []
    for player in soup_list:
        position = player.find('div', style='font-size: .8rem;')
        position_list.append(str(position.text))
    final_position = []
    for position in position_list:
        position = position.strip("\n")
        position = position.strip(" ")
        final_position.append(position[0:2].strip(" "))
    return final_position

In [107]:
link_soup = Soup_Generator(savant_page_list[0:20])

In [108]:
#Verify first 20:
Position_Puller(link_soup)

['C',
 '3B',
 'CF',
 '3B',
 '3B',
 'RF',
 'CF',
 '1B',
 '1B',
 '2B',
 'CF',
 '1B',
 '1B',
 'CF',
 '3B',
 'C',
 '1B',
 'C',
 'RF',
 'CF']

### Trying to Merge FanGraphs wRC+ (target stat) data with above dataframe:

Link to use:
https://www.fangraphs.com/leaders/season-stat-grid?position=B&seasonStart=2015&seasonEnd=2020&stat=wRC%2B&pastMinPt=200&curMinPt=0&mode=normal

Importing all functions from my fangraphs_wrc_code.py file:

In [49]:
from fangraphs_wrc_code import *

In [50]:
fg_url = 'https://www.fangraphs.com/leaders/season-stat-grid?position=B&seasonStart=2015&seasonEnd=2020&stat=wRC%2B&pastMinPt=200&curMinPt=0&mode=normal'

Using the wRC_DataFrame_shifter function to pull the wRC+ dataframe from the URL above:

In [51]:
wrc_df = wRC_DataFrame_Shifter(fg_url)

Spitting out a sample of 15 to make sure everything is working:

In [52]:
wrc_df.sample(15)

,Name-Year,wRC+
2840,Ildemaro Vargas-2017,0
199,Mike Yastrzemski-2016,0
2721,Daniel Murphy-2018,110
4980,Seth Smith-2015,113
325,Clint Frazier-2016,0
2177,Lewis Brinson-2020,73
1618,Ben Gamel-2019,87
4062,Jonny Gomes-2015,84
1454,Luis Torrens-2017,0
2997,Logan Forsythe-2018,71


It's here!  Next Step: need to use name-year as an ID to match with the baseball savant dataframe

Renaming column for later merging:

In [68]:
wrc_df.rename(columns={'Name-Year':'Player_Year_ID'}, inplace=True)

In [69]:
wrc_df.head(2)

,Player_Year_ID,wRC+
0,Michael Lorenzen-2015,0
1,Michael Lorenzen-2016,0


## Merging Baseball Savant and FanGraphs Data:

Reminding myself how the savant dataframe looks:

In [53]:
savant_df.head()

,Player,Year,Age,PA,G,Avg EV (MPH),Avg LA (°),Barrel%,Solid Contact %,Hard Hit %,Zone Swing %,Out of Zone Swing %,Pull %,Straight Away %,Oppo %,GB%,FB%,LD %,Sprint Speed,Player Link
Rk.,,,,,,,,,,,,,,,,,,,,
1,"Saltalamacchia, Jarrod",2016.0,31.0,292.0,92.0,89.2,21.4,11.6,6.8,35.4,65.7,23.9,44.2,34.0,21.8,31.3,43.5,19.7,25.6,[/savant-player/457454]
2,"Schimpf, Ryan",2016.0,28.0,330.0,89.0,90.3,29.7,16.7,8.0,41.4,64.7,23.4,41.4,33.9,24.7,20.1,42.0,21.8,28.2,[/savant-player/572114]
3,"Gallo, Joey",2017.0,24.0,532.0,145.0,93.3,23.0,22.1,8.3,52.2,72.7,28.2,50.2,29.2,20.6,29.2,37.9,22.1,27.8,[/savant-player/608336]
4,"Carpenter, Matt",2018.0,33.0,677.0,156.0,90.7,21.6,13.7,11.5,44.7,57.1,18.5,47.6,30.7,21.7,28.3,37.8,28.5,26.5,[/savant-player/572761]
5,"Carpenter, Matt",2017.0,32.0,622.0,145.0,89.9,22.7,8.2,8.4,37.7,55.1,14.0,45.9,31.4,22.7,29.0,37.7,26.6,26.6,[/savant-player/572761]


First, handling player names:

In [54]:
player_list = savant_df.Player.tolist()

In [56]:
player_list_split = [player.split(', ') for player in player_list]
for player in player_list_split:
    player[0] = player[0].strip()

final_player_list = [(player[1] + ' ' + player[0]) for player in player_list_split]    

Verify:

In [57]:
print(final_player_list)

['Jarrod Saltalamacchia', 'Ryan Schimpf', 'Joey Gallo', 'Matt Carpenter', 'Matt Carpenter', 'Jay Bruce', 'Joey Gallo', 'Eric Thames', 'Rhys Hoskins', 'Cavan Biggio', 'Greg Bird', 'Joey Gallo', 'Mike Napoli', 'Mike Trout', 'Kyle Seager', 'Mitch Garver', 'Eric Thames', 'Stephen Vogt', 'Jay Bruce', 'Brandon Belt', 'Jake Marisnick', 'Robinson Chirinos', 'Nick Castellanos', 'Matt Carpenter', 'Adam Duvall', 'Mike Trout', 'Chris Davis', 'Rhys Hoskins', 'Miguel Sano', 'Chris Carter', 'Adam Duvall', 'Robinson Chirinos', 'Justin Turner', 'Matt Carpenter', 'Luis Valbuena', 'Lucas Duda', 'Rhys Hoskins', 'Colby Rasmus', 'Chris Carter', 'Max Muncy', 'Kyle Seager', 'Eduardo Escobar', 'Brandon Belt', 'Mike Trout', 'Austin Riley', 'Travis Shaw', 'Miguel Sano', 'Aaron Judge', 'Freddie Freeman', 'Brandon Moss', 'Colby Rasmus', 'Curtis Granderson', 'Joey Votto', 'Luis Valbuena', 'Jay Bruce', 'Hunter Renfroe', 'Khris Davis', 'Tom Murphy', 'Christin Stewart', 'Gary Sanchez', 'Kyle Seager', 'ByungHo Park', '

Next: years

In [62]:
year_list = savant_df.Year.tolist()

In [64]:
year_list = [int(year) for year in year_list]
print(year_list)

[2016, 2016, 2017, 2018, 2017, 2019, 2018, 2018, 2017, 2019, 2018, 2019, 2017, 2019, 2017, 2019, 2019, 2019, 2018, 2018, 2017, 2017, 2016, 2019, 2018, 2020, 2016, 2018, 2018, 2017, 2020, 2018, 2017, 2016, 2018, 2017, 2019, 2016, 2015, 2018, 2020, 2019, 2019, 2018, 2019, 2019, 2016, 2017, 2017, 2017, 2015, 2019, 2020, 2017, 2017, 2019, 2018, 2019, 2019, 2019, 2019, 2016, 2016, 2018, 2016, 2019, 2016, 2017, 2018, 2017, 2015, 2018, 2018, 2018, 2017, 2019, 2015, 2018, 2017, 2017, 2017, 2017, 2019, 2018, 2019, 2019, 2016, 2019, 2018, 2017, 2019, 2019, 2016, 2016, 2018, 2018, 2017, 2019, 2019, 2019, 2015, 2016, 2016, 2016, 2018, 2018, 2018, 2017, 2019, 2017, 2017, 2016, 2016, 2016, 2019, 2015, 2016, 2018, 2017, 2017, 2016, 2015, 2017, 2019, 2019, 2018, 2018, 2017, 2017, 2017, 2019, 2019, 2015, 2018, 2018, 2020, 2017, 2019, 2019, 2015, 2015, 2016, 2020, 2016, 2018, 2018, 2019, 2019, 2016, 2016, 2018, 2017, 2019, 2016, 2017, 2017, 2019, 2016, 2020, 2019, 2019, 2019, 2019, 2019, 2015, 2018, 201

Great!  Now for merging the two:

In [65]:
player_year_id = []
for i, player in enumerate(final_player_list):
    id_str = player + "-" + str(year_list[i])
    player_year_id.append(id_str)
print(player_year_id)

['Jarrod Saltalamacchia-2016', 'Ryan Schimpf-2016', 'Joey Gallo-2017', 'Matt Carpenter-2018', 'Matt Carpenter-2017', 'Jay Bruce-2019', 'Joey Gallo-2018', 'Eric Thames-2018', 'Rhys Hoskins-2017', 'Cavan Biggio-2019', 'Greg Bird-2018', 'Joey Gallo-2019', 'Mike Napoli-2017', 'Mike Trout-2019', 'Kyle Seager-2017', 'Mitch Garver-2019', 'Eric Thames-2019', 'Stephen Vogt-2019', 'Jay Bruce-2018', 'Brandon Belt-2018', 'Jake Marisnick-2017', 'Robinson Chirinos-2017', 'Nick Castellanos-2016', 'Matt Carpenter-2019', 'Adam Duvall-2018', 'Mike Trout-2020', 'Chris Davis-2016', 'Rhys Hoskins-2018', 'Miguel Sano-2018', 'Chris Carter-2017', 'Adam Duvall-2020', 'Robinson Chirinos-2018', 'Justin Turner-2017', 'Matt Carpenter-2016', 'Luis Valbuena-2018', 'Lucas Duda-2017', 'Rhys Hoskins-2019', 'Colby Rasmus-2016', 'Chris Carter-2015', 'Max Muncy-2018', 'Kyle Seager-2020', 'Eduardo Escobar-2019', 'Brandon Belt-2019', 'Mike Trout-2018', 'Austin Riley-2019', 'Travis Shaw-2019', 'Miguel Sano-2016', 'Aaron Judg

Perfect, next to add as a column to the savant df:

In [66]:
savant_df['Player_Year_ID'] = player_year_id

In [67]:
savant_df.sample(15)

,Player,Year,Age,PA,G,Avg EV (MPH),Avg LA (°),Barrel%,Solid Contact %,Hard Hit %,...,Out of Zone Swing %,Pull %,Straight Away %,Oppo %,GB%,FB%,LD %,Sprint Speed,Player Link,Player_Year_ID
Rk.,,,,,,,,,,,,,,,,,,,,,
210,"Duvall, Adam",2017.0,29.0,647.0,157.0,87.3,21.0,8.9,5.6,32.7,...,31.4,42.8,31.5,25.7,33.4,29.0,24.3,27.7,[/savant-player/594807],Adam Duvall-2017
367,"Harrison, Josh",2018.0,31.0,374.0,97.0,85.9,15.7,3.9,7.1,26.1,...,36.0,41.3,32.9,25.8,39.2,26.9,26.5,27.1,[/savant-player/543281],Josh Harrison-2018
1286,"Santana, Domingo",2017.0,25.0,607.0,151.0,89.2,10.5,9.4,6.0,40.4,...,24.2,33.4,38.6,28.0,46.6,19.7,31.1,27.2,[/savant-player/570267],Domingo Santana-2017
148,"Votto, Joey",2019.0,36.0,608.0,142.0,88.8,15.3,6.9,5.7,38.2,...,18.5,35.3,39.5,25.2,37.8,30.1,28.9,25.0,[/savant-player/458015],Joey Votto-2019
583,"McCutchen, Andrew",2019.0,33.0,262.0,59.0,90.6,12.7,7.3,5.5,40.9,...,14.5,43.3,36.0,20.7,45.7,25.0,23.2,28.8,[/savant-player/457705],Andrew McCutchen-2019
1698,"Martin, Richie",2019.0,25.0,309.0,120.0,83.2,9.4,2.4,3.9,25.2,...,34.9,31.1,43.2,25.7,52.4,16.0,21.8,29.5,[/savant-player/621006],Richie Martin-2019
1474,"Cuddyer, Michael",2015.0,36.0,408.0,117.0,87.5,7.8,4.5,6.2,30.8,...,35.2,38.0,32.5,29.5,50.3,18.2,25.3,27.2,[/savant-player/150212],Michael Cuddyer-2015
1658,"Martinez, Jose",2019.0,31.0,373.0,128.0,88.9,6.9,7.1,8.7,39.3,...,26.6,26.8,44.5,28.7,50.4,16.5,29.9,26.2,[/savant-player/500874],Jose Martinez-2019
209,"Suzuki, Kurt",2018.0,35.0,388.0,105.0,87.2,18.2,3.9,6.8,35.0,...,33.7,47.7,33.9,18.4,36.5,29.0,24.8,25.6,[/savant-player/435559],Kurt Suzuki-2018


Perfect!  They all match in the random 15 line item sample.  Final step: loading the wRC+ data into the same dataframe.

In [81]:
combined_df = savant_df.merge(wrc_df, how='left', on='Player_Year_ID')

In [82]:
combined_df

,Player,Year,Age,PA,G,Avg EV (MPH),Avg LA (°),Barrel%,Solid Contact %,Hard Hit %,...,Pull %,Straight Away %,Oppo %,GB%,FB%,LD %,Sprint Speed,Player Link,Player_Year_ID,wRC+
0,"Saltalamacchia, Jarrod",2016.0,31.0,292.0,92.0,89.2,21.4,11.6,6.8,35.4,...,44.2,34.0,21.8,31.3,43.5,19.7,25.6,[/savant-player/457454],Jarrod Saltalamacchia-2016,70
1,"Schimpf, Ryan",2016.0,28.0,330.0,89.0,90.3,29.7,16.7,8.0,41.4,...,41.4,33.9,24.7,20.1,42.0,21.8,28.2,[/savant-player/572114],Ryan Schimpf-2016,128
2,"Gallo, Joey",2017.0,24.0,532.0,145.0,93.3,23.0,22.1,8.3,52.2,...,50.2,29.2,20.6,29.2,37.9,22.1,27.8,[/savant-player/608336],Joey Gallo-2017,119
3,"Carpenter, Matt",2018.0,33.0,677.0,156.0,90.7,21.6,13.7,11.5,44.7,...,47.6,30.7,21.7,28.3,37.8,28.5,26.5,[/savant-player/572761],Matt Carpenter-2018,140
4,"Carpenter, Matt",2017.0,32.0,622.0,145.0,89.9,22.7,8.2,8.4,37.7,...,45.9,31.4,22.7,29.0,37.7,26.6,26.6,[/savant-player/572761],Matt Carpenter-2017,124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1888,"Jankowski, Travis",2016.0,25.0,383.0,131.0,86.2,0.8,1.3,1.3,26.9,...,20.2,43.3,36.6,62.2,10.1,26.5,29.5,[/savant-player/608671],Travis Jankowski-2016,81
1889,"LeMahieu, DJ",2016.0,28.0,635.0,146.0,91.7,5.9,4.9,5.6,47.5,...,20.8,42.0,37.2,52.9,10.1,33.7,27.1,[/savant-player/518934],DJ LeMahieu-2016,130
1890,"Jankowski, Travis",2018.0,27.0,387.0,117.0,85.6,3.3,0.7,1.1,19.3,...,30.1,39.9,30.1,60.1,10.1,25.4,29.0,[/savant-player/608671],Travis Jankowski-2018,90
1891,"Slater, Austin",2018.0,26.0,225.0,74.0,87.3,2.6,2.3,0.8,35.1,...,23.7,36.6,39.7,63.4,9.2,22.9,28.0,[/savant-player/596103],Austin Slater-2018,81


In [83]:
savant_df

,Player,Year,Age,PA,G,Avg EV (MPH),Avg LA (°),Barrel%,Solid Contact %,Hard Hit %,...,Out of Zone Swing %,Pull %,Straight Away %,Oppo %,GB%,FB%,LD %,Sprint Speed,Player Link,Player_Year_ID
Rk.,,,,,,,,,,,,,,,,,,,,,
1,"Saltalamacchia, Jarrod",2016.0,31.0,292.0,92.0,89.2,21.4,11.6,6.8,35.4,...,23.9,44.2,34.0,21.8,31.3,43.5,19.7,25.6,[/savant-player/457454],Jarrod Saltalamacchia-2016
2,"Schimpf, Ryan",2016.0,28.0,330.0,89.0,90.3,29.7,16.7,8.0,41.4,...,23.4,41.4,33.9,24.7,20.1,42.0,21.8,28.2,[/savant-player/572114],Ryan Schimpf-2016
3,"Gallo, Joey",2017.0,24.0,532.0,145.0,93.3,23.0,22.1,8.3,52.2,...,28.2,50.2,29.2,20.6,29.2,37.9,22.1,27.8,[/savant-player/608336],Joey Gallo-2017
4,"Carpenter, Matt",2018.0,33.0,677.0,156.0,90.7,21.6,13.7,11.5,44.7,...,18.5,47.6,30.7,21.7,28.3,37.8,28.5,26.5,[/savant-player/572761],Matt Carpenter-2018
5,"Carpenter, Matt",2017.0,32.0,622.0,145.0,89.9,22.7,8.2,8.4,37.7,...,14.0,45.9,31.4,22.7,29.0,37.7,26.6,26.6,[/savant-player/572761],Matt Carpenter-2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1889,"Jankowski, Travis",2016.0,25.0,383.0,131.0,86.2,0.8,1.3,1.3,26.9,...,21.4,20.2,43.3,36.6,62.2,10.1,26.5,29.5,[/savant-player/608671],Travis Jankowski-2016
1890,"LeMahieu, DJ",2016.0,28.0,635.0,146.0,91.7,5.9,4.9,5.6,47.5,...,23.0,20.8,42.0,37.2,52.9,10.1,33.7,27.1,[/savant-player/518934],DJ LeMahieu-2016
1891,"Jankowski, Travis",2018.0,27.0,387.0,117.0,85.6,3.3,0.7,1.1,19.3,...,18.7,30.1,39.9,30.1,60.1,10.1,25.4,29.0,[/savant-player/608671],Travis Jankowski-2018


Same number of rows with the added column!  Join is complete.

### Now, initial exploring of combined dataframe:

In [89]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1893 entries, 0 to 1892
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Player               1893 non-null   object 
 1   Year                 1893 non-null   float64
 2   Age                  1893 non-null   float64
 3   PA                   1893 non-null   float64
 4   G                    1893 non-null   float64
 5   Avg EV (MPH)         1893 non-null   float64
 6   Avg LA (°)           1893 non-null   float64
 7   Barrel%              1893 non-null   float64
 8   Solid Contact %      1893 non-null   float64
 9   Hard Hit %           1893 non-null   float64
 10  Zone Swing %         1893 non-null   float64
 11  Out of Zone Swing %  1893 non-null   float64
 12  Pull %               1893 non-null   float64
 13  Straight Away %      1893 non-null   float64
 14  Oppo %               1893 non-null   float64
 15  GB%                  1893 non-null   f

Good news: no null values, besides wRC+.  Bad news: over 200 Null data points in wRC+... let's see why that is

In [90]:
print(combined_df[combined_df['wRC+'].isnull()])

                   Player    Year   Age     PA      G  Avg EV (MPH)  \
28           Sano, Miguel  2018.0  25.0  299.0   71.0          90.3   
46           Sano, Miguel  2016.0  23.0  495.0  116.0          92.4   
59          Sanchez, Gary  2019.0  27.0  446.0  106.0          91.2   
65           Sano, Miguel  2019.0  26.0  439.0  105.0          94.4   
75     Hernandez, Teoscar  2019.0  27.0  464.0  125.0          91.2   
...                   ...     ...   ...    ...    ...           ...   
1845       Nunez, Eduardo  2015.0  28.0  204.0   72.0          88.7   
1854        Garcia, Leury  2018.0  27.0  275.0   82.0          88.0   
1858       Blanco, Andres  2016.0  32.0  209.0   90.0          88.5   
1874       Iglesias, Jose  2015.0  25.0  454.0  120.0          83.8   
1879       Tomas, Yasmany  2015.0  25.0  426.0  118.0          89.3   

      Avg LA (°)  Barrel%  Solid Contact %  Hard Hit %  ...  Pull %  \
28          12.6     11.1              8.5        44.8  ...    37.3   
46   

Ah... Accent marks in the name (i.e. Miguel Sano should be Sanó, Gary Sanchez should be Sánchez, Teoscar Hernandez should be Hernández, Yasmany Tomas should be Tomás, etc.).  Let's see how many different players have null values:

In [94]:
players_to_fix = combined_df.Player[combined_df['wRC+'].isnull()].drop_duplicates()

In [96]:
players_to_fix.to_list()

[' Sano, Miguel',
 ' Sanchez, Gary',
 ' Hernandez, Teoscar',
 ' Aguilar, Jesus',
 ' Cespedes, Yoenis',
 ' Suarez, Eugenio',
 ' Diaz, Isan',
 ' Rivera, Rene',
 ' Ramirez, Jose',
 ' Encarnacion, Edwin',
 ' Acuna Jr., Ronald',
 ' Pina, Manny',
 ' Gomez, Carlos',
 ' Narvaez, Omar',
 ' Hechavarria, Adeiny',
 ' Cabrera, Asdrubal',
 ' Bautista, Jose',
 ' Nunez, Renato',
 ' Hernandez, Enrique',
 ' Laureano, Ramon',
 ' Moncada, Yoan',
 ' Rodriguez, Ronny',
 ' Leon, Sandy',
 ' Beltran, Carlos',
 ' Mejia, Francisco',
 ' Diaz, Aledmys',
 ' Peraza, Jose',
 ' Beltre, Adrian',
 ' Maldonado, Martin',
 ' Vazquez, Christian',
 ' Gonzalez, Adrian',
 ' Perez, Roberto',
 ' Garcia, Avisail',
 ' Urias, Luis',
 ' Guzman, Ronald',
 ' Perez, Hernan',
 ' Jimenez, Eloy',
 ' Osuna, Jose',
 ' Pagan, Angel',
 ' Reyes, Jose',
 ' Andujar, Miguel',
 ' Sanchez, Yolmer',
 ' Baez, Javier',
 ' Perez, Carlos',
 ' Martinez, Jose',
 ' Gonzalez, Carlos',
 ' Diaz, Elias',
 ' Herrera, Odubel',
 ' Rodriguez, Sean',
 ' Tejada, Rub

In [97]:
len(players_to_fix)

68

OK, 68 players to fix...

### **Next step:** is there a better way to do this besides brute force?